<a href="https://colab.research.google.com/github/uzaramen108/fenicsx-colab/blob/main/workshop/fenics_workshop_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# --------------------------------------------------
# 1️⃣ Mount Google Drive (optional, for cache)
# --------------------------------------------------
from google.colab import drive
import os

if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")
else:
    print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    #print("$", " ".join(map(str, cmd)))
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

# ==================================================
# 4️⃣ Sanity check
# ==================================================
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

📦 Google Drive already mounted
📦 Repository already exists — skipping clone
🚀 Running setup_fenicsx.py in current kernel
📦 Google Drive detected — using persistent cache
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENMPI
MPI version        : mpiexec (OpenRTE) 4.1.2
MPI ranks (-np)    : 4


In [36]:
%%fenicsx -np 4 --time
from mpi4py import MPI
import dolfinx
from dolfinx import io
import numpy as np

def save_separate_meshes(N_coarse: int, N_fine: int, degree: int):
    comm = MPI.COMM_WORLD

    # 공통: 타겟 함수 정의
    def g(x):
        return x[0] + np.sin(np.pi * x[0]) * np.cos(3 * np.pi * x[0])

    # -------------------------------------------------------
    # 1. 듬성듬성한 메쉬 (Coarse) -> 파란색 꺾은선 그래프용
    # -------------------------------------------------------
    mesh_c = dolfinx.mesh.create_unit_interval(comm, N_coarse)
    V_c = dolfinx.fem.functionspace(mesh_c, ("Lagrange", degree))

    u_approx = dolfinx.fem.Function(V_c)
    u_approx.interpolate(g)
    u_approx.name = "Approximation_Coarse"

    with io.XDMFFile(comm, "result_coarse.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as xdmf:
        xdmf.write_mesh(mesh_c)
        xdmf.write_function(u_approx)

    # -------------------------------------------------------
    # 2. 아주 촘촘한 메쉬 (Fine) -> 빨간색 부드러운 곡선용
    # -------------------------------------------------------
    mesh_f = dolfinx.mesh.create_unit_interval(comm, N_fine)
    V_f = dolfinx.fem.functionspace(mesh_f, ("Lagrange", degree))

    u_exact = dolfinx.fem.Function(V_f)
    u_exact.interpolate(g)
    u_exact.name = "Exact_Fine"

    with io.XDMFFile(comm, "result_fine.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as xdmf:
        xdmf.write_mesh(mesh_f)
        xdmf.write_function(u_exact)

# 실행: 근사값은 5개 구간, 정답은 200개 구간으로 계산
save_separate_meshes(N_coarse=10, N_fine=200, degree=1)

if MPI.COMM_WORLD.rank == 0:
    print("\n✅ 두 개의 파일이 생성되었습니다.")
    print("   1. result_coarse.xdmf (파란색용)")
    print("   2. result_fine.xdmf (빨간색용)")


✅ 두 개의 파일이 생성되었습니다.
   1. result_coarse.xdmf (파란색용)
   2. result_fine.xdmf (빨간색용)
⏱ Elapsed time: 2.015961 s


In [35]:
import os
import glob
import shutil

def clean_workspace():
    print("🧹 작업 공간 청소를 시작합니다...")

    # 1. 지울 파일 확장자 목록 (필요하면 여기에 더 추가하게)
    extensions_to_delete = [
        "*.xdmf", "*.h5",       # FEniCSx 결과
        "*.vtu", "*.pvd",       # ParaView 데이터
        "*.zip",                # 압축 파일
        "*.png", "*.jpg",       # 이미지 파일
        "*.json"                # 기타 데이터
    ]

    # 2. 지울 폴더 이름 목록
    folders_to_delete = [
        "out_pyvista",          # 아까 만들었던 PyVista 출력 폴더
        "__pycache__"           # 파이썬 캐시 폴더
    ]

    # --- 파일 삭제 루프 ---
    deleted_files_count = 0
    for pattern in extensions_to_delete:
        # 현재 경로에서 해당 패턴의 파일들을 모두 찾음
        files = glob.glob(pattern)
        for file in files:
            try:
                os.remove(file)
                deleted_files_count += 1
                # print(f"삭제됨: {file}") # 너무 많이 출력되면 주석 처리
            except OSError as e:
                print(f"❌ 삭제 실패: {file} - {e}")

    # --- 폴더 삭제 루프 ---
    deleted_folders_count = 0
    for folder in folders_to_delete:
        if os.path.exists(folder):
            try:
                shutil.rmtree(folder) # 폴더 내부까지 싹 지움
                deleted_folders_count += 1
                print(f"🗑️ 폴더 삭제됨: {folder}/")
            except OSError as e:
                print(f"❌ 폴더 삭제 실패: {folder} - {e}")

    print("-" * 30)
    print(f"✨ 청소 완료! 총 {deleted_files_count}개의 파일과 {deleted_folders_count}개의 폴더를 삭제했습니다.")
    print("   이제 작업 공간이 깨끗합니다.")

# 함수 실행
clean_workspace()

🧹 작업 공간 청소를 시작합니다...
------------------------------
✨ 청소 완료! 총 4개의 파일과 0개의 폴더를 삭제했습니다.
   이제 작업 공간이 깨끗합니다.


In [ ]:
# Define domain

mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 100, 100, cell_type=dolfinx.mesh.CellType.quadrilateral)
Q = dolfinx.fem.functionspace(mesh, ("DG", 0))
c = dolfinx.fem.Function(Q)
c.x.array[:] = 0.1
cells_left = dolfinx.mesh.locate_entities(mesh, mesh.topology.dim, lambda x: x[0] <= 0.7 + 1e-12)
c.x.array[cells_left] = 0.01
c.x.scatter_forward()

c_plotter = pyvista.Plotter()
c_mesh = pyvista.UnstructuredGrid(*dolfinx.plot.vtk_mesh(mesh))
c_mesh.cell_data["c"] = c.x.array
c_plotter.add_mesh(c_mesh)
c_plotter.show()

In [ ]:
%%fenicsx -np 4 --time
from mpi4py import MPI
import dolfinx
from dolfinx import io
import numpy as np
import ufl

mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 100, 100, cell_type=dolfinx.mesh.CellType.quadrilateral) #동일
Q = dolfinx.fem.functionspace(mesh, ("DG", 0)) #동일
c = dolfinx.fem.Function(Q) #동일
c.x.array[:] = 0.1 # 동일
cells_left = dolfinx.mesh.locate_entities(mesh, mesh.topology.dim, lambda x: x[0] <= 0.7 + 1e-12) # 동일
c.x.array[cells_left] = 0.01 # 동일
c.x.scatter_forward() # 동일
c.name = "Conductivity_c"
filename = "conductivity_data.xdmf"

with io.XDMFFile(mesh.comm, filename, "w", encoding=io.XDMFFile.Encoding.HDF5) as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(c)

if mesh.comm.rank == 0:
    print(f"\n✅ 저장이 완료되었습니다: {filename}")
    print("👉 ParaView에서 열 때 주의: 이 데이터는 'Cell Data'입니다.")

In [ ]:
%%fenicsx -np 4 --time
from mpi4py import MPI
from petsc4py import PETSc
import dolfinx
from dolfinx import io
from dolfinx.fem import petsc
import ufl
import numpy as np

# 1. 도메인 및 재료 물성 정의
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 100, 100, cell_type=dolfinx.mesh.CellType.quadrilateral)

# 재료 계수 c (DG0)
Q = dolfinx.fem.functionspace(mesh, ("DG", 0))
c = dolfinx.fem.Function(Q)
c.name = "Conductivity_c"

c.x.array[:] = 0.1
cells_left = dolfinx.mesh.locate_entities(mesh, mesh.topology.dim, lambda x: x[0] <= 0.7 + 1e-12)
c.x.array[cells_left] = 0.01
c.x.scatter_forward()

# 2. 열전달 문제 정의
V = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
T = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

T_n = dolfinx.fem.Function(V)
T_n.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]))
T_n.name = "Temperature"

dt = dolfinx.fem.Constant(mesh, dolfinx.default_scalar_type(0.01))
g = dolfinx.fem.Function(V)
g.x.array[:] = 0
f = dolfinx.fem.Function(V)
f.x.array[:] = 0

mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
boundary_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
boundary_dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, boundary_facets)
bcs = [dolfinx.fem.dirichletbc(g, boundary_dofs)]

F = (
    ufl.inner(T - T_n, v) * ufl.dx
    + dt * c * ufl.inner(ufl.grad(T), ufl.grad(v)) * ufl.dx
    - dt * ufl.inner(f, v) * ufl.dx
)
a, L = dolfinx.fem.form(ufl.system(F))

# 3. 선형대수 구조 생성
A = petsc.create_matrix(a)

# -------------------------------------------------------------------------
# [수정됨] b 벡터 생성 방식 변경
# L 대신 행렬 A의 구조를 이용해 벡터를 생성하면 오류가 사라집니다.
# -------------------------------------------------------------------------
b = A.createVecRight()

# Solver 설정
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

# 행렬 조립
A.zeroEntries()
petsc.assemble_matrix(A, a, bcs=bcs)
A.assemble()

# 4. 시간 루프 및 파일 저장
t = 0
T_end = 0.1
Th = dolfinx.fem.Function(V)
Th.x.array[:] = T_n.x.array
Th.name = "Temperature"

xdmf_T = io.XDMFFile(mesh.comm, "heat_result.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5)
xdmf_c = io.XDMFFile(mesh.comm, "conductivity.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5)

xdmf_T.write_mesh(mesh)
xdmf_c.write_mesh(mesh)

xdmf_T.write_function(Th, t)
xdmf_c.write_function(c)
xdmf_c.close()

while t < T_end:
    t += float(dt)

    with b.localForm() as loc_b:
        loc_b.set(0)

    petsc.assemble_vector(b, L)

    petsc.apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    petsc.set_bc(b, bcs)

    solver.solve(b, Th.vector)
    Th.x.scatter_forward()

    T_n.x.array[:] = Th.x.array[:]
    xdmf_T.write_function(Th, t)

xdmf_T.close()

if MPI.COMM_WORLD.rank == 0:
    print("\n✅ 오류 해결 완료!")
    print("   1. conductivity.xdmf (재료 물성 c)")
    print("   2. heat_result.xdmf (온도 변화 T)")